In [1]:
pip install praw

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import praw
# Authorized instance
reddit_authorized = praw.Reddit(client_id="DlcjXhOBwypg_mZEm6xlwA",         # your client id
                                client_secret="kLA5U-4G-eKi47VoCgunW1bnd2lf9Q",      # your client secret
                                user_agent="Project",        # your user agent
                                username="blackbear199710",        # your reddit username
                                password="CWJcwj1010")        # your reddit password

reddit_read_only = praw.Reddit(client_id="DlcjXhOBwypg_mZEm6xlwA",         # your client id
                               client_secret="kLA5U-4G-eKi47VoCgunW1bnd2lf9Q",      # your client secret
                               user_agent="Project")        # your user agent

subreddit = reddit_read_only.subreddit("wallstreetbets")
 
# Display the name of the Subreddit
print("Display Name:", subreddit.display_name)
 
# Display the title of the Subreddit
print("Title:", subreddit.title)
 
# Display the description of the Subreddit
print("Description:", subreddit.description)

Display Name: wallstreetbets
Title: wallstreetbets
Description: Our official Twitter: [@Official_WSB](https://twitter.com/official_wsb)

---

The rules and submission guidelines are maintained on new Reddit so be sure to check them and make sure you're up to date.

* Read the [rules](https://www.reddit.com/r/wallstreetbets/about/rules)

* Read the [comment and submission guide](https://www.reddit.com/r/wallstreetbets/wiki/contentguide)

* Read the [FAQ](https://www.reddit.com/r/wallstreetbets/wiki/faq) if you're new to both wallstreetbets and trading.

---
**Join the discord**

[WSB Discord](https://discord.gg/rncPUDCJ)

**filter by flairs**

^Navigate ^WSB |^We ^recommend ^best ^daily ^DD
:--|:--     
**DD** | [All](https://ns.reddit.com/r/wallstreetbets/search?sort=new&restrict_sr=on&q=flair%3ADD) / [**Best Daily**](https://ns.reddit.com/r/wallstreetbets/search?sort=top&q=flair%3ADD&restrict_sr=on&t=day) / [Best Weekly](https://ns.reddit.com/r/wallstreetbets/search?sort=top&q=flair%3

In [27]:
def posts_collection(stock, sort, time):
    
    posts_dict = {"Title": [], "Post Text": [],
              "ID": [], "Score": [],
              "Total Comments": [], "Post URL": []
              }
    for name in stock:
        for post in reddit_read_only.subreddit("wallstreetbets").search(name,sort=sort,time_filter=time):
    # Title of each post
            posts_dict["Title"].append(post.title)
     
    # Text inside a post
            posts_dict["Post Text"].append(post.selftext)
     
    # Unique ID of each post
            posts_dict["ID"].append(post.id)
     
    # The score of a post
            posts_dict["Score"].append(post.score)
     
    # Total number of comments inside the post
            posts_dict["Total Comments"].append(post.num_comments)
     
    # URL of each post
            posts_dict["Post URL"].append(post.url)


    # Save posts as dataframe 
    stock_posts = pd.DataFrame(posts_dict)
    return stock_posts

In [60]:
tesla = ['tesla','TSLA']
apple = ['apple','AAPL']
amazon = ['amazon','AMZN']
google = ['google','GOOG']
meta = ['facebook','META']
netfilx = ['netfilx','NFLX']
time = 'year'

# Collect posts for all six companies

Tesla_posts = posts_collection(tesla,"relevance", time)
Apple_posts = posts_collection(apple,"relevance", time)
Amazon_posts = posts_collection(amazon,"relevance", time)
Google_posts = posts_collection(google,"relevance", time)
Meta_posts = posts_collection(meta,"relevance", time)
Netfilx_posts = posts_collection(netfilx,"top", time)

In [34]:
ids = Apple_posts["ID"]
a = Apple_posts[ids.isin(ids[ids.duplicated()])].sort_values("ID")
len(a)

30

In [61]:
print('The number of data point in the Apple_posts is',len(Apple_posts))
Apple_posts_clean = Apple_posts.drop_duplicates(subset=['Title'])
print('The number of data point after removing duplicates is',len(Apple_posts_clean))
ids = Apple_posts_clean["ID"]
a = Apple_posts_clean[ids.isin(ids[ids.duplicated()])].sort_values("ID")
print('The number of duplicates after cleaning up',len(a))
Apple_posts_clean.to_csv("Apple_posts_clean.csv", index=True)

The number of data point in the Apple_posts is 200
The number of data point after removing duplicates is 194
The number of duplicates after cleaning up 0


In [62]:
print('The number of data point in the Tesla_posts is',len(Tesla_posts))
Tesla_posts_clean = Tesla_posts.drop_duplicates(subset=['Title'])
print('The number of data point after removing duplicates is',len(Tesla_posts_clean))
ids = Tesla_posts_clean["ID"]
a = Tesla_posts_clean[ids.isin(ids[ids.duplicated()])].sort_values("ID")
print('The number of duplicates after cleaning up',len(a))
Tesla_posts_clean.to_csv("Tesla_posts_clean.csv", index=True)

The number of data point in the Tesla_posts is 200
The number of data point after removing duplicates is 198
The number of duplicates after cleaning up 0


In [70]:
print('The number of data point in the Meta_posts is',len(Meta_posts))
Meta_posts_clean = Meta_posts.drop_duplicates(subset=['Title'])
print('The number of data point after removing duplicates is',len(Meta_posts_clean))
ids = Meta_posts_clean["ID"]
a = Meta_posts_clean[ids.isin(ids[ids.duplicated()])].sort_values("ID")
print('The number of duplicates after cleaning up',len(a))
Meta_posts_clean.to_csv("Meta_posts_clean.csv", index=True)

The number of data point in the Meta_posts is 200
The number of data point after removing duplicates is 185
The number of duplicates after cleaning up 0


In [71]:
print('The number of data point in the Amazon_posts is',len(Amazon_posts))
Amazon_posts_clean = Amazon_posts.drop_duplicates(subset=['Title'])
print('The number of data point after removing duplicates is',len(Amazon_posts_clean))
ids = Amazon_posts_clean["ID"]
a = Amazon_posts_clean[ids.isin(ids[ids.duplicated()])].sort_values("ID")
print('The number of duplicates after cleaning up',len(a))
Amazon_posts_clean.to_csv("Amazon_posts_clean.csv", index=True)

The number of data point in the Amazon_posts is 200
The number of data point after removing duplicates is 193
The number of duplicates after cleaning up 0


In [72]:
print('The number of data point in the Google_posts is',len(Google_posts))
Google_posts_clean = Google_posts.drop_duplicates(subset=['Title'])
print('The number of data point after removing duplicates is',len(Google_posts_clean))
ids = Google_posts_clean["ID"]
a = Google_posts_clean[ids.isin(ids[ids.duplicated()])].sort_values("ID")
print('The number of duplicates after cleaning up',len(a))
Google_posts_clean.to_csv("Google_posts_clean.csv", index=True)

The number of data point in the Google_posts is 200
The number of data point after removing duplicates is 197
The number of duplicates after cleaning up 0


In [73]:
print('The number of data point in the Google_posts is',len(Netfilx_posts))
Netfilx_posts_clean = Netfilx_posts.drop_duplicates(subset=['Title'])
print('The number of data point after removing duplicates is',len(Netfilx_posts_clean))
ids = Netfilx_posts_clean["ID"]
a = Netfilx_posts_clean[ids.isin(ids[ids.duplicated()])].sort_values("ID")
print('The number of duplicates after cleaning up',len(a))
Netfilx_posts_clean.to_csv("Netfilx_posts_clean.csv", index=True)

The number of data point in the Google_posts is 100
The number of data point after removing duplicates is 100
The number of duplicates after cleaning up 0
